In [17]:

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import tensorflow as tf  # Version 1.0.0 (some previous versions are used in past commits)
from sklearn import metrics
import random
from random import randint
import time
import os
import numpy as np
from collections import namedtuple
import cv2
from pathlib import Path
import argparse
import os
from openvino.inference_engine import IENetwork, IECore
import csv
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix
from tensorflow import keras
import statistics

In [8]:
# data = 원본데이터 | data2 = 우리가 측정한 데이터
folder_path_1 = "./data/이상(방화)"
folder_path_2 = "./data/이상(유기)"
folder_path_3 = "./data/이상(전도)"
folder_path_4 = "./data/가방"
folder_path_5 = "./data/이상(파손)"
folder_path_6 = "./data/이상(폭행)"
folder_path_7 = "./data/이상(흡연)"
#folder_path_7 = "./data/이상(교통약자)"
folder_path_8 = "./data/구매(구매)"
folder_path_9 = "./data/구매(반품)"
folder_path_10 = "./data/구매(비교)"
folder_path_11 = "./data/구매(선택)"
folder_path_12 = "./data/주머니"

In [6]:
def concat(path):
    file_lst = os.listdir(path)
    df_list = []
    for file in file_lst:
        file_name = os.path.join(path, file)
        df = pd.read_csv(file_name)
        df = df[df.NUMOFBODIES != 0].reset_index(drop=True)
# ### 인덱스 크기 확인하려면 주석
#         if len(df) < 610:
#             pad_length = 610 - len(df)
#             pad_df = pd.DataFrame(0, index=range(len(df), 610), columns=df.columns)
#             df = pd.concat([df, pad_df])
#         df['index_num'] = df.index
# ###
        df_list.append(df)
    #concat_df = pd.concat(df_list, ignore_index=True)
    return df_list

In [9]:
fire_mer = concat(folder_path_1)
yugi_mer = concat(folder_path_2)
jeon_mer = concat(folder_path_3)
gabang_mer = concat(folder_path_4)
damage_mer = concat(folder_path_5)
violence_mer = concat(folder_path_6)
smoke_mer = concat(folder_path_7)
buy_mer = concat(folder_path_8)
refund_mer = concat(folder_path_9)
compar_mer = concat(folder_path_10)
select_mer = concat(folder_path_11)
jumoney_mer = concat(folder_path_12)

In [10]:
mer_lst = [fire_mer,
            yugi_mer,
            jeon_mer,
            jumoney_mer,
            gabang_mer,
            damage_mer,
            violence_mer,
            smoke_mer,
            buy_mer,
            refund_mer,
            compar_mer,
            select_mer]

In [11]:
def lengs(name):
    lengths = []
    for i in range(len(name)):
        lengths.append(len(name[i]))
    return lengths

In [12]:
lengs(mer_lst)

[733, 722, 726, 191, 437, 722, 722, 772, 244, 253, 247, 287]

In [21]:
lengs(smoke_mer)

[165,
 161,
 158,
 158,
 131,
 138,
 135,
 140,
 150,
 154,
 151,
 154,
 316,
 144,
 141,
 146,
 134,
 135,
 140,
 163,
 160,
 157,
 132,
 146,
 150,
 149,
 152,
 152,
 157,
 152,
 158,
 318,
 163,
 159,
 164,
 138,
 146,
 154,
 154,
 151,
 139,
 142,
 147,
 150,
 139,
 145,
 141,
 147,
 148,
 152,
 148,
 154,
 293,
 129,
 125,
 89,
 165,
 161,
 159,
 113,
 115,
 114,
 118,
 117,
 147,
 152,
 151,
 156,
 152,
 155,
 157,
 322,
 157,
 154,
 157,
 155,
 153,
 155,
 139,
 138,
 138,
 144,
 141,
 147,
 145,
 150,
 137,
 142,
 138,
 143,
 332,
 157,
 154,
 157,
 137,
 142,
 162,
 156,
 159,
 109,
 111,
 111,
 111,
 111,
 143,
 149,
 147,
 152,
 150,
 153,
 150,
 155,
 287,
 126,
 125,
 140,
 139,
 145,
 162,
 155,
 159,
 139,
 141,
 141,
 141,
 142,
 133,
 141,
 138,
 143,
 140,
 144,
 140,
 146,
 302,
 129,
 126,
 129,
 122,
 123,
 126,
 166,
 164,
 162,
 144,
 142,
 144,
 151,
 150,
 121,
 127,
 123,
 129,
 147,
 151,
 149,
 151,
 294,
 118,
 114,
 118,
 115,
 119,
 162,
 153,
 159,
 120,

In [ ]:
lengs()

In [ ]:
avg

In [19]:
lengsed = []
for i in mer_lst:
    lengsed.append(statistics.mean(lengs(i)))
lengsedb

[140.38199181446112,
 130.27285318559558,
 160.0082644628099,
 155.55497382198953,
 150.49199084668192,
 139.39612188365652,
 268.33795013850414,
 152.05310880829015,
 416.41803278688525,
 408.02766798418975,
 443.9068825910931,
 329.94425087108016]

In [22]:
lengsed = []
for i in mer_lst:
    lengsed.append(max(lengs(i)))
lengsed

[313, 310, 308, 326, 309, 315, 484, 352, 540, 556, 550, 502]

In [ ]:
buy_mer = pd.concat(buy_mer, ignore_index=True)
refund_mer = pd.concat(refund_mer, ignore_index=True)
compar_mer = pd.concat(compar_mer, ignore_index=True)
select_mer = pd.concat(select_mer, ignore_index=True)
#test_mer = pd.concat(test_mer, ignore_index=True)

In [ ]:
buy_mer['LABEL'] = 0  
refund_mer['LABEL'] = 1  
compar_mer['LABEL'] = 2
select_mer['LABEL'] = 3
#test_mer['LABEL'] = 4

In [ ]:
category_lst = [buy_mer, refund_mer, compar_mer, select_mer] # test_mer
full_data = pd.concat(category_lst, ignore_index=True)
full_data.shape

In [ ]:
full_data.columns

In [ ]:
# 데이터와 레이블을 추출합니다
label_data = full_data['LABEL']
data_without_label = full_data.drop(['ID', 'TIMESTAMP', 'FRAME_NUM', 'LABEL'], axis=1)

In [ ]:
sequence_start_indices = full_data.index[full_data['index_num'] == 0].tolist()
sequence_lengths = [sequence_start_indices[i] - sequence_start_indices[i-1] for i in range(1, len(sequence_start_indices))]
sequence_lengths.insert(0, sequence_start_indices[0])

# Min sequence length
min_sequence_length = min(sequence_lengths)

In [ ]:
min_sequence_length = 610

In [ ]:
X_sequences = []
y_sequences = []

# Iterating over the start indices to create sequences
for i in range(len(sequence_start_indices) - 1):
    start_index = sequence_start_indices[i]
    end_index = sequence_start_indices[i + 1]
    
    # Ensure the sequence has the expected length before appending
    if end_index - start_index == min_sequence_length:
        X_sequence = data_without_label.iloc[start_index:end_index].values
        y_sequence = label_data.iloc[start_index:end_index].values
        X_sequences.append(X_sequence)
        y_sequences.append(y_sequence[-1])  # Taking the label of the last frame for the sequence

# Converting lists to numpy arrays
X_sequences = np.array(X_sequences)
y_sequences = np.array(y_sequences)

X_sequences.shape, y_sequences.shape


In [ ]:
import tensorflow as tf
from sklearn.model_selection import train_test_split

X_train, X_temp, y_train, y_temp = train_test_split(X_sequences, y_sequences, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.3, random_state=42)

In [ ]:
print(X_train.shape, y_train.shape)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# 모델 정의
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(128, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])),
    tf.keras.layers.LSTM(64, return_sequences=True),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(4, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 훈련 데이터에 대한 샘플 가중치 마스크 생성
# 패딩이 아닌 값은 1, 패딩된 값은 0
sample_weight_train = np.where(X_train != 0, 1, 0)
sample_weight_train = sample_weight_train.max(axis=-1)

# 각 시퀀스에 대한 평균 샘플 가중치 계산
sample_weight_train_avg = sample_weight_train.mean(axis=1)

In [ ]:
X_train = X_train.astype(np.float32)
y_train = y_train.astype(np.float32)
X_val = X_val.astype(np.float32)
y_val = y_val.astype(np.float32)
X_test = X_test.astype(np.float32)
y_test = y_test.astype(np.float32)


In [ ]:
model_file_path = "./model/best_lstm_model_tttt.h5"
checkpoint_cb = keras.callbacks.ModelCheckpoint(model_file_path)
# - 자동 훈련 멈추기 함수 사용 : 추가 훈련 epoch 2회, 가중치 업데이트
early_stopping_cb = keras.callbacks.EarlyStopping(patience=10,
                                        restore_best_weights=True)


# 4. Model Training
# 모델 훈련
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=1000,
                    batch_size=128, sample_weight=sample_weight_train_avg,
                    callbacks=[checkpoint_cb, early_stopping_cb])

# 5. Model Evaluation
# 모델 평가
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy*100:.2f}%")


In [ ]:
from keras.models import load_model
model1 = load_model('./model/best_lstm_model_tttt.h5')

In [ ]:
model1.summary()

In [ ]:
### 불러온 모델로 데이터 정확도 확인하기
# - 예측
test_preds = model1.predict(X_val)
test_preds.shape

In [ ]:

# - 정답갯수 / 오답 갯수 (100개 데이터에 대해서만)
# - 정답률 / 오답률 (100개 데이터에 대해서만)
test_pred_idx = [np.argmax(pred) for pred in test_preds]
test_pred_idx

all_cnt = len(test_pred_idx)
o_cnt = len(y_val[(test_pred_idx == y_val)])
x_cnt = len(y_val[(test_pred_idx != y_val)])

print("정답갯수 : {}개, 오답갯수 : {}개".format(o_cnt, x_cnt))
print("정답률 : {}%, 오답률 : {}%".format(o_cnt/all_cnt*100,
                                        x_cnt/all_cnt*100))

In [ ]:
cate = ['구매(구매)', '구매(반품)', '구매(비교)', '구매(선택)']#'구매(시험)',

In [ ]:
# 유니크한 종속변수들을 얻습니다.
unique_targets = np.unique(y_val)

# 테스트 데이터에 대한 예측을 한 번만 수행
test_preds = model1.predict(X_val)
test_pred_idx = [np.argmax(pred) for pred in test_preds]

In [ ]:
np.unique(test_pred_idx)

In [ ]:
test_pred_idx = np.array(test_pred_idx)

In [ ]:
# 각 종속변수별 정확도를 저장할 데이터프레임
accuracy_per_target = pd.DataFrame(index=unique_targets, columns=['accuracy'])

In [ ]:
# 각 클래스별 정확도 계산
class_accuracies = {}
for i in np.unique(y_val):
    idx = np.where(y_val == i)
    correct = np.sum(test_pred_idx[idx] == y_val[idx])
    total = len(idx[0])
    class_accuracies[i] = correct / total
    accuracy_per_target.loc[i, 'accuracy'] = class_accuracies[i]

In [ ]:
accuracy_per_target

In [ ]:
accuracy_per_target["Target"] = cate

In [ ]:
accuracy_df_sorted = accuracy_per_target.sort_values(by='accuracy', ascending=True)

In [ ]:
accuracy_df_sorted.head(20)